In [61]:
import camelot
import json
import os
import re

In [62]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [ ]:
hello = get_biggest_table("D:/study/LammaIndex/documents/Chuong_V_Yeu_cau_ky_thuat.pdf",15)

In [6]:
data = hello["data"]

In [7]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [ ]:
converted_data

In [44]:
context_queries = {}  # Dict chứa thông tin chi tiết theo key
product_key = {}  # Dict lồng: ten_san_pham -> ten_hang_hoa -> list[key]

for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                q = value[0]
                k = value[1]
                value_str = ' '.join(value)
            else:
                q = None
                k = value
                value_str = value

            # Ghi vào context_queries
            context_queries[key] = {
                "ten_san_pham": ten_san_pham,
                "ten_hang_hoa": ten_hang_hoa,
                "value": value_str,
                "yeu_cau_ky_thuat_chi_tiet": k,
                "yeu_cau_ky_thuat": q
            }

            # Ghi vào product_key
            if ten_san_pham not in product_key:
                product_key[ten_san_pham] = {}
            if ten_hang_hoa not in product_key[ten_san_pham]:
                product_key[ten_san_pham][ten_hang_hoa] = []
            product_key[ten_san_pham][ten_hang_hoa].append(key)



In [ ]:
context_queries

In [ ]:
product_key

In [ ]:
# Lưu với tên file mặc định
with open('context_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(context_prompts, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu {len(context_prompts)} context prompts vào file: context_prompts.json")

In [47]:
def prompt_create_query(context_prompt):
    prompt =  f"""
    Bạn sẽ nhận được một đoạn mô tả kỹ thuật ngắn gọn (context_prompt), thường là các mảnh thông tin kỹ thuật rời rạc. Nhiệm vụ của bạn là chuyển đổi đoạn mô tả đó thành một truy vấn (query) hoàn chỉnh bằng ngôn ngữ tự nhiên, với mục tiêu:

- Diễn đạt lại đoạn mô tả dưới dạng một câu hỏi hoặc câu truy vấn rõ ràng, dễ hiểu.
- Giữ nguyên đầy đủ tất cả các thông tin kỹ thuật có trong câu gốc (chế độ, dòng điện, điện áp, bước công suất,...).
- Không được loại bỏ hoặc làm mờ bất kỳ chi tiết kỹ thuật nào.
- Truy vấn cần phù hợp để tìm kiếm thông tin tương đồng về mặt ngữ nghĩa trong một hệ thống truy xuất dữ liệu kỹ thuật.

Dưới đây là các ví dụ:

**Input:** "Tải giả xả acquy Thông số kỹ thuật Dòng xả lớn nhất với dải điện áp 220V DC ÷ 240V DC – tương ứng với 1 bộ ≥ 70 A"

**Output:** "Dòng xả lớn nhất của tải giả xả acquy là bao nhiêu khi dải điện áp nằm trong khoảng 220V DC đến 240V DC, và tương ứng với 1 bộ thì có đạt dòng xả ≥ 70A không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Các chế độ xả - Gồm 4 chế độ: dòng không đổi, công suất không đổi, theo đặc tính dòng cho trước, điều chỉnh thủ công."

**Output:** "Tải giả xả acquy hỗ trợ những chế độ xả nào? Có phải gồm dòng không đổi, công suất không đổi, theo đặc tính dòng định trước, và điều chỉnh thủ công không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Bước công suất điều chỉnh xả tải 100 W"

**Output:** "Bước điều chỉnh công suất xả tải nhỏ nhất của tải giả xả acquy là bao nhiêu? Có phải là 100W không?"

---
Trả lời từng `context_prompt` theo cấu trúc như trên.


    DỮ LIỆU ĐẦU VÀO:
    ---
    {context_prompt}
    ---

    CHỈ CẦN TRẢ VỀ CÂU TRUY VẤN.
    """
    return prompt

In [48]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
test = dict(list(context_queries.items())[70:75])
test

In [82]:
for key in test:
    data = test[key] 
    prompt = prompt_create_query(f"{data['ten_san_pham']} {data['ten_hang_hoa']} {data['value']}")
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        temperature=0
    )
    output_text = response.output_text.strip()
    test[key]["query"]=output_text

In [ ]:
test, product_key

In [84]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)

# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini")
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="thong_tin_san_pham",
    client=client,
    aclient=aclient,
)

def retrieve_document(query_str):
    file_names = []
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_document = MetadataFilters(
        filters=[
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="summary_document"),
        ],
    condition=FilterCondition.AND,
    )
    retriever_document = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_document)
    
    results = retriever_document.retrieve(query_str)

    for result in results:
        metadata = result.metadata
        file_names.append(metadata["file_name"])

    return file_names

def retrieve_chunk(file_names, query_str):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_chunk = MetadataFilters(
        filters=[
            MetadataFilter(key="file_name", operator=FilterOperator.IN, value=file_names),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
        ],
        condition=FilterCondition.AND,
    )

    retriever_chunk = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_chunk)
    
    results = retriever_chunk.retrieve(query_str)
    content = ""
    for i, result in enumerate(results, start=1):
        metadata = result.metadata
        file_name = metadata["file_name"]+ ".pdf"
        page = metadata["page"]
        table = metadata["table_name"]
        figure_name = metadata.get("figure_name")
        text = result.text.strip()
        content += f"Chunk {i} trong file {file_name} tại trang {page}, có chứa bảng {table} và hình {figure_name} có nội dung:\n{text}\n\n"

    return content


        




In [85]:
for product in product_key:
    items = product_key[product]
    for key in items:
        for item in items[key]:
            if item not in test:
                continue
            query = test[item]["query"]
            content = retrieve_chunk(file_names, query)
            test[item]["content"] = content            
            

In [86]:
for product in product_key:
    file_names = retrieve_document(product)
    items = product_key[product]
    for item in items:
        if item not in test:
            continue
        query = test[item]["query"]
        content = retrieve_chunk(file_names, query)
        test[item]["content"] = content

In [87]:
for key in test:
    print(key)

5F1A0
97D75
49234
59016
3CD8F


In [88]:
SYSTEM_PROMPT = """
Bạn được cung cấp:
- Một hoặc nhiều đoạn văn bản (chunk) từ tài liệu kỹ thuật, kèm metadata: tên file, mục, bảng/hình (nếu có), số trang
- Một yêu cầu kỹ thuật cụ thể.
- Một đoạn văn mẫu.

#Yêu cầu trả lời bằng tiếng việt:
# 1. Tìm thông tin kỹ thuật liên quan trực tiếp đến yêu cầu kỹ thuật.
# 2. Trích xuất giá trị thông số để xác định khả năng đáp ứng theo yêu cầu và trả về đoạn văn tương tự giống đoạn văn mẫu không thêm bớt nhưng thông số phải chính xác trong tài liệu.
# 3. Dẫn chứng rõ: file, section, table/figure name (nếu có), page, nội dung trích dẫn của những tài liệu liên quan, những tài liệu khác không liên quan thì bỏ qua.

# #Output: JSON gồm các trường:
- yeu_cau_ky_thuat
- kha_nang_dap_ung
- tai_lieu_tham_chieu" 

# Ví dụ:
Input:
Yêu cầu: "Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4"  
Chunk: "...NetSure 731 A41-S8: 4 rectifier slots (standard), expandable to 6..."  
Metadata:  
- file: "Netsure-731-A41-user-manual.pdf"  
- section: "Table 1-1 Configuration of power system"  
- page: 2"
Đoạn văn mẫu: Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4 (ví dụ tìm trong tài liệu số lượng là 5 thì trả về "Số lượng khe cắm module chỉnh lưu (Rectifier): 5")
"""

# Định nghĩa function schema
FUNCTION_SCHEMA = {
    "name": "danh_gia_ky_thuat",
    "description": "Đánh giá khả năng đáp ứng của sản phẩm theo yêu cầu kỹ thuật từ chunk tài liệu.",
    "parameters": {
        "type": "object",
        "properties": {
            "yeu_cau_ky_thuat": {"type": "string"},
            "kha_nang_dap_ung": {"type": "string"},
            "tai_lieu_tham_chieu": {
                "type": "object",
                "properties": {
                    "file": {"type": "string"},
                    "section": {"type": "string"},
                    "table_or_figure": {"type": "string"},
                    "page": {"type": "integer"},
                    "evidence": {"type": "string"}
                },
                "required": ["file", "section", "page", "evidence"]
            }
        },
        "required": ["yeu_cau_ky_thuat", "kha_nang_dap_ung", "tai_lieu_tham_chieu"]
    }
}

In [89]:
# Tạo assistant
# === CREATE ASSISTANT ===
def create_assistant():
    assistant = client.beta.assistants.create(
        name="Technical Document Evaluator",
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === CREATE THREAD ===
def create_thread():
    thread = client.beta.threads.create()
    return thread.id

# === UPDATE ASSISTANT ===
def update_assistant(assistant_id):
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === EVALUATE TECHNICAL REQUIREMENT ===
def evaluate_technical_requirement(user_prompt, assistant_id):
    # 1. Tạo thread riêng cho mỗi lần gọi
    thread = client.beta.threads.create()
    thread_id = thread.id

    # 2. Gửi message vào thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_prompt
    )

    # 3. Tạo run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )

    # 4. Chờ assistant xử lý (tối đa 20s)
    for _ in range(20):
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        if run.status not in ["queued", "in_progress"]:
            break
        time.sleep(1)

    # 5. Lấy arguments trực tiếp
    if run.status == "requires_action":
        call = run.required_action.submit_tool_outputs.tool_calls[0]
        print(f"👉 Assistant đã gọi tool: {call.function.name}")
        print("🧠 Dữ liệu JSON assistant muốn trả về:")
        print(call.function.arguments)
        return call.function.arguments

    elif run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            print(f"[{msg.role}] {msg.content[0].text.value}")
        return None

    else:
        print(f"Run status: {run.status}")
        return None



In [90]:
test

{'5F1A0': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'yeu_cau_ky_thuat_chi_tiet': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'yeu_cau_ky_thuat': None,
  'query': 'Bộ chuyển đổi nguồn 220VAC/48VDC kèm theo 02 dàn acquy 200Ah có thời gian bảo hành là bao lâu? Có phải theo tiêu chuẩn của nhà sản xuất, tối thiểu là 12 tháng không?',
  'content': 'Chunk 1 trong file NetSure_732_User_Manual.pdf tại trang 29, có chứa bảng Technical and Engineering Data for NetSure 732 Subrack Power System và hình None có nội dung:\nIt can withstand five times of simulated lightning surge current of 20Ka at 8/20μs, for the positive and negative polarities respectively. The test interval is not smaller than 1 minute. It can also withstand one event of simulated lightning surge current of 40kA at 8/20μs. |\n\n\nNe

In [91]:
import openai
from openai import OpenAI
import json
client = OpenAI()

In [92]:
# Ví dụ sử dụng
import time
assistant_id = create_assistant()
print(f"Assistant ID: {assistant_id}")


# Ví dụ user prompt

for key in test:
    if not test[key].get('content'):
        continue
    data = test[key]
    value = data['value']
    content = data['content']
    form = data['yeu_cau_ky_thuat_chi_tiet']
    # test[key].pop("content", None)
    
    user_prompt = f'''
    Chunk và metadata: {content}
    Yêu cầu: {value}
    Đoạn văn mẫu: {form}
    '''
    
    # Gọi hàm đánh giá với thread riêng
    result = evaluate_technical_requirement(user_prompt, assistant_id)
    test[key]['response'] = result




Assistant ID: asst_sRiLQPBCsGOii4EAPGi49Nml


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22860\985798817.py:30: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22860\985798817.py:34: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22860\985798817.py:41: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22860\985798817.py:49: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)


👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.","kha_nang_dap_ung":"Thời gian bảo hành của sản phẩm là theo tiêu chuẩn của nhà sản xuất.","tai_lieu_tham_chieu":{"file":"NetSure_732_User_Manual.pdf","section":"Appendix 1  Technical And Engineering Data","table_or_figure":"None","page":29,"evidence":"Thời gian bảo hành của sản phẩm là theo tiêu chuẩn của nhà sản xuất."}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4","kha_nang_dap_ung":"Số lượng khe cắm module chỉnh lưu (Rectifier): 4","tai_lieu_tham_chieu":{"file":"NetSure_732_User_Manual.pdf","section":"Table 1-1 Configuration of power system","page":8,"evidence":"Maximum configuration：4 pieces"}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Công s

In [93]:
for key in test:
    test[key].pop("content", None)
    response = test[key]['response']
    if isinstance(response, str):
        response = json.loads(response)
    test[key]["kha_nang_dap_ung"] = response['kha_nang_dap_ung']
    test[key]["tai_lieu_tham_chieu"] = {
            "file": response['tai_lieu_tham_chieu']['file'],
            "section": response['tai_lieu_tham_chieu'].get('section', ''),
            "table_or_figure": response['tai_lieu_tham_chieu'].get('table_or_figure', ''),
            "page": response['tai_lieu_tham_chieu'].get('page', 0),
            "evidence": response['tai_lieu_tham_chieu'].get('evidence', '')
        }
    test[key].pop("response", None)

In [96]:
for product in product_key:
    items = product_key[product]
    for key in items:
        for item in items[key]:
            if item not in test:
                continue
            query = test[item]["query"]
            content = retrieve_chunk(file_names, query)
            test[item]["content"] = content 

({'Tải giả xả acquy': {'Yêu cầu chung': ['9F506', '656FA', '665C0', '146FA'],
   'Thông số kỹ thuật': ['C315E',
    'C8261',
    'F46D0',
    '2A2FB',
    'C3465',
    '4A8FB',
    '96B0B',
    '54BC9',
    '8699B',
    '83051',
    'F2306',
    '329B9',
    'E7263',
    '6E7E6',
    '76366',
    'DC048',
    'F830C',
    '7B53A',
    '4A38E',
    '98209',
    '8C8F1',
    '7DD5E',
    '888BC',
    '8481C',
    '5B7E9',
    '2394F',
    '44934',
    '5F7A6',
    'BA835',
    '649E2',
    'BF81A',
    '2D1BC',
    'EEE2A',
    'A57D8',
    '18E7C',
    '97A68',
    '85D16',
    '8D24F',
    'D108B',
    '7453C']},
  'Đồng hồ đo nội trở acquy': {'Yêu cầu chung': ['C0D49', '5E94A', 'BA0A9'],
   'Các tính năng': ['359DC', '98335', 'EA16D'],
   'Đặc tính kỹ thuật': ['C0973', '1BD01', '7F6DD'],
   'Thân máy': ['AFE62', '72955', '6A67B', 'D999D'],
   'Pin': ['2DF59', '6F170'],
   'Phụ kiện': ['CC377', '7E288', '24351', '32B8A', '550A3']},
  'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn 

In [122]:
def prompt_adapt_or_not(dap_ung_ky_thuat: str, tai_lieu_tham_chieu: str) -> str:
    prompt = f"""
Tôi có một danh sách các thông số kỹ thuật và khả năng đáp ứng tương ứng, mỗi dòng được ngăn cách bằng '||' theo định dạng:

    yêu cầu kỹ thuật || khả năng đáp ứng

Dưới đây là danh sách đầu vào:
{dap_ung_ky_thuat}

Ngoài ra, tôi cũng có các tài liệu tham chiếu như sau:
{tai_lieu_tham_chieu}

Yêu cầu của bạn là:
1. Kiểm tra xem khả năng đáp ứng có thỏa mãn từng yêu cầu kỹ thuật hay không.
2. Nếu tất cả các yêu cầu đều được đáp ứng, trả về "đáp ứng", ngược lại trả về "không đáp ứng".
3. Gộp lại các tài liệu tham chiếu bị trùng để tránh lặp lại(ví dụ như tên file, trang, bảng).

Chỉ trả về kết quả theo định dạng JSON như sau:

{{
    "đáp ứng kỹ thuật": "Đáp ứng" hoặc "Không đáp ứng",
    "tài liệu": "danh sách tài liệu đã rút gọn viết dưới dạng txt"
}}
"""
    return prompt

In [104]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [119]:
def parse_output_text(output_text: str) -> dict:
    # B1: Loại bỏ phần ```json ... ```
    cleaned = re.sub(r"^```json\n|```$", "", output_text.strip())

    # B2: Giải mã các ký tự escape như \n, \"
    unescaped = cleaned.encode("utf-8")

    # B3: Chuyển thành dict
    return json.loads(unescaped)

In [120]:
results = []
for product in product_key:
    items = product_key[product]
    for key in items:
        dap_ung_ky_thuat = ""
        tai_lieu_tham_chieu = ""
        for item in items[key]:
            if item not in test:
                continue
            yeu_cau_ky_thuat = test[item]['yeu_cau_ky_thuat_chi_tiet']
            kha_nang_dap_ung = test[item]['kha_nang_dap_ung']
            dap_ung_ky_thuat += f"{yeu_cau_ky_thuat} || {kha_nang_dap_ung}\n"
            tai_lieu = test[item]['tai_lieu_tham_chieu']
            tai_lieu_tham_chieu += f"file {tai_lieu['file']},trang: {tai_lieu['page']}, trong bảng(figure):{tai_lieu['table_or_figure']},evidence :{tai_lieu['evidence']} \n"
        if dap_ung_ky_thuat != "" and tai_lieu_tham_chieu != "":
            prompt = prompt_adapt_or_not(dap_ung_ky_thuat, tai_lieu_tham_chieu)
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                temperature=0
            )
            output_text = response.output_text.strip()
            output_text = parse_output_text(output_text)
            print(output_text)
            product_key[product][key].append(output_text['đáp ứng kỹ thuật'])
            product_key[product][key].append(output_text['tài liệu'])

{'đáp ứng kỹ thuật': 'đáp ứng', 'tài liệu': 'file NetSure_732_User_Manual.pdf, trang: 29'}
{'đáp ứng kỹ thuật': 'đáp ứng', 'tài liệu': 'file NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure):, evidence: Maximum configuration: 4 pieces; file Converter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Output Power: 3000W Maximum; file NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure): Table 1-1, evidence: DC distribution | PL: | 32A/1P×2;16A/1P×2 MCB | NPL: 63A/1P×2;32A/1P×4;16A/1P×2 MCB'}


In [121]:
product_key 

{'Tải giả xả acquy': {'Yêu cầu chung': ['9F506', '656FA', '665C0', '146FA'],
  'Thông số kỹ thuật': ['C315E',
   'C8261',
   'F46D0',
   '2A2FB',
   'C3465',
   '4A8FB',
   '96B0B',
   '54BC9',
   '8699B',
   '83051',
   'F2306',
   '329B9',
   'E7263',
   '6E7E6',
   '76366',
   'DC048',
   'F830C',
   '7B53A',
   '4A38E',
   '98209',
   '8C8F1',
   '7DD5E',
   '888BC',
   '8481C',
   '5B7E9',
   '2394F',
   '44934',
   '5F7A6',
   'BA835',
   '649E2',
   'BF81A',
   '2D1BC',
   'EEE2A',
   'A57D8',
   '18E7C',
   '97A68',
   '85D16',
   '8D24F',
   'D108B',
   '7453C']},
 'Đồng hồ đo nội trở acquy': {'Yêu cầu chung': ['C0D49', '5E94A', 'BA0A9'],
  'Các tính năng': ['359DC', '98335', 'EA16D'],
  'Đặc tính kỹ thuật': ['C0973', '1BD01', '7F6DD'],
  'Thân máy': ['AFE62', '72955', '6A67B', 'D999D'],
  'Pin': ['2DF59', '6F170'],
  'Phụ kiện': ['CC377', '7E288', '24351', '32B8A', '550A3']},
 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['DF25C',
   '2

In [123]:
test

{'5F1A0': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'yeu_cau_ky_thuat_chi_tiet': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'yeu_cau_ky_thuat': None,
  'query': 'Bộ chuyển đổi nguồn 220VAC/48VDC kèm theo 02 dàn acquy 200Ah có thời gian bảo hành là bao lâu? Có phải theo tiêu chuẩn của nhà sản xuất, tối thiểu là 12 tháng không?',
  'kha_nang_dap_ung': 'Thời gian bảo hành của sản phẩm là theo tiêu chuẩn của nhà sản xuất.',
  'tai_lieu_tham_chieu': {'file': 'NetSure_732_User_Manual.pdf',
   'section': 'Appendix 1  Technical And Engineering Data',
   'table_or_figure': 'None',
   'page': 29,
   'evidence': 'Thời gian bảo hành của sản phẩm là theo tiêu chuẩn của nhà sản xuất.'}},
 '97D75': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'C

In [124]:
product_key_test = {
    'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)':{
        'Yêu cầu chung': ['5F1A0','đáp ứng', 'file NetSure_732_User_Manual.pdf, trang: 29'],
        'Cấu hình thiết bị nguồn': ['97D75','49234','59016','đáp ứng','file NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure):, evidence: Maximum configuration: 4 pieces; file Converter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Output Power: 3000W Maximum; file NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure): Table 1-1, evidence: DC distribution | PL: | 32A/1P×2;16A/1P×2 MCB | NPL: 63A/1P×2;32A/1P×4;16A/1P×2 MCB'],
    }
}

In [125]:
product_key_test

{'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['5F1A0',
   'đáp ứng',
   'file NetSure_732_User_Manual.pdf, trang: 29'],
  'Cấu hình thiết bị nguồn': ['97D75',
   '49234',
   '59016',
   'đáp ứng',
   'file NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure):, evidence: Maximum configuration: 4 pieces; file Converter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Output Power: 3000W Maximum; file NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure): Table 1-1, evidence: DC distribution | PL: | 32A/1P×2;16A/1P×2 MCB | NPL: 63A/1P×2;32A/1P×4;16A/1P×2 MCB']}}

In [126]:
test

{'5F1A0': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'yeu_cau_ky_thuat_chi_tiet': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.',
  'yeu_cau_ky_thuat': None,
  'query': 'Bộ chuyển đổi nguồn 220VAC/48VDC kèm theo 02 dàn acquy 200Ah có thời gian bảo hành là bao lâu? Có phải theo tiêu chuẩn của nhà sản xuất, tối thiểu là 12 tháng không?',
  'kha_nang_dap_ung': 'Thời gian bảo hành của sản phẩm là theo tiêu chuẩn của nhà sản xuất.',
  'tai_lieu_tham_chieu': {'file': 'NetSure_732_User_Manual.pdf',
   'section': 'Appendix 1  Technical And Engineering Data',
   'table_or_figure': 'None',
   'page': 29,
   'evidence': 'Thời gian bảo hành của sản phẩm là theo tiêu chuẩn của nhà sản xuất.'}},
 '97D75': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'C

In [127]:
!pip install python-docx

In [128]:
from docx import Document
from docx.shared import Pt
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
# === Tạo Word document ===
doc = Document()
doc.add_heading("BẢNG TUYÊN BỐ ĐÁP ỨNG VỀ KỸ THUẬT", level=1)

# Tạo bảng 6 cột
table = doc.add_table(rows=1, cols=6)
table.style = 'Table Grid'
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Header
headers = [
    "Hạng mục số", "Tên hàng hoá",
    "Thông số kỹ thuật và các tiêu chuẩn của hàng hoá trong E-HSMT",
    "Thông số kỹ thuật và các tiêu chuẩn của hàng hoá chào trong E-HSDT",
    "Hồ sơ tham chiếu", "Tình đáp ứng của hàng hoá"
]

for i, text in enumerate(headers):
    cell = table.rows[0].cells[i]
    cell.text = text
    for p in cell.paragraphs:
        for run in p.runs:
            run.font.bold = True
            run.font.size = Pt(10)
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# Ghi từng dòng
for product, hang_hoa_dict in product_key_test.items():
    for idx, (ten_hang_hoa, items) in enumerate(hang_hoa_dict.items(), start=1):
        ma_ids = items[:-2]  # Các ID
        dap_ung = items[-2]  # ví dụ: "đáp ứng"
        ho_so = items[-1]    # tài liệu tham chiếu

        # Tổng hợp thông số kỹ thuật
        eh_smt = ""
        eh_hsdt = ""

        for ma in ma_ids:
            if ma in test:
                eh_smt += f"- {test[ma]['yeu_cau_ky_thuat_chi_tiet']}\n"
                eh_hsdt += f"- {test[ma]['kha_nang_dap_ung']}\n"

        # Tạo dòng mới
        row = table.add_row().cells
        row[0].text = str(idx)
        row[1].text = ten_hang_hoa
        row[2].text = eh_smt.strip()
        row[3].text = eh_hsdt.strip()
        row[4].text = ho_so
        row[5].text = dap_ung

        for cell in row:
            cell.vertical_alignment = WD_ALIGN_VERTICAL.TOP
            for p in cell.paragraphs:
                for run in p.runs:
                    run.font.size = Pt(9)

# Lưu file
doc.save("bang_tuyen_bo_dap_ung.docx")